In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
# RSS 뉴스 URL
news_rss_url = {
    '매일경제' : 'https://www.mk.co.kr/rss/50200011/',
    '한국경제' : 'https://www.hankyung.com/feed/finance' 
}

In [4]:
# 기사 1건의 정보를 저장하기위한 자료구조
class Article :
    def __init__(self, *, media, datetime, title, content):
        self.media = media         # 언론사명
        self.datetime = datetime   # 작성일시
        self.title = title         # 뉴스제목
        self.content = content     # 뉴스내용

In [6]:
# 뉴스기사 수집하기
def get_news_rss(media, url) :   
    """doc-string
    매개변수: media-언론사명, url-언론사의 rss url
    반환값 : 기사를 요소로 갖는 목록
    """
    article_list = [] # 기사 목록 (Ariticle객체를 요소로 갖는 리스트)
    
    title_list = []  # 기사 제목 목록
    link_list = []   # 기사 링크 목록
    news_data = []   # 기사 본문 목록
    news_datetime = [] # 기사 작성일시 목록

    # 수집봇이 아닌 브라우져가 접근한것 처럼 요청하기위함
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
    }    
    news_rss = requests.get(url,headers)
    if news_rss.status_code != 200 :
        raise Exception('응답오류 발생')
        
    news_rss_soup = BeautifulSoup(news_rss.content,'xml')
    # 기사 제목 
    title_list = news_rss_soup.select('item > title')
    title_list = [title.text for title in title_list]
    # 기사 상세링크 
    link_list = news_rss_soup.select('item > link')
    link_list = [link.text for link in link_list]
    # 기사 내용, 기사 작성일시
    for link in link_list :
        news_res = requests.get(link, headers=headers)
        news_content_soup = BeautifulSoup(news_res.content, 'lxml')

        try:
            if media == '매일경제':
                news_content = news_content_soup.select_one('.news_cnt_detail_wrap')
                news_data.append(news_content.text)
                news_datetime.append(news_content_soup.select_one('.time_area .registration dd').text)    
            elif media == '한국경제':
                news_content = news_content_soup.select_one('#articletxt')
                news_data.append(news_content.text)
                news_datetime.append(news_content_soup.select_one('.datetime > .item > .txt-date').text)    
        except Exception as e:
            print(len(news_data),link)
            print(e)
            
    # print(title_list)
    # print(news_data)
    # print(news_datetime)
    print(len(title_list), len(news_data), len(news_datetime))
    for item in zip(title_list, news_data, news_datetime):
        # print(media, item[0],item[1],item[2])
        article = Article(media = media, title=item[0], content=item[1], datetime=item[2])
        article_list.append(article)    
    
    return article_list

In [7]:
# 모든 언론사 기사를 합치기
all_article_list = []
for media, url in news_rss_url.items():
    print(media,url)
    all_article_list.append(get_news_rss(media,url))
print(len(all_article_list))

매일경제 https://www.mk.co.kr/rss/50200011/
50 50 50
한국경제 https://www.hankyung.com/feed/finance
50 50 50
2


In [9]:
result = np.array(all_article_list).flatten().tolist()
print(len(result))
print(type(result))

100
<class 'list'>


In [10]:
data = {
    '언론사' : [ article.media for article in result ],
    '제목' : [ article.title for article in result ],
    '내용' : [ article.content for article in result ],
    '작성일시' : [ article.datetime for article in result ],
}

In [11]:
df = pd.DataFrame(data)

In [12]:
df.head(3)

,언론사,제목,내용,작성일시
0,매일경제,‘오징어게임2’ 외신 혹평에...테마주 동반 폭락,\n 사진 확대 ‘오징어 게임2’. 제공|넷플릭스 전 세계적 기대작...,2024-12-27 09:38:05
1,매일경제,주가는 뒷걸음질인데 외국인은 산다?…‘게임주’에 쏠리는 시선,\n 사진 확대 독일에서 열린 게임쇼 ‘게임스컴’ 부스의 크래프톤 로...,2024-12-27 09:00:51
2,매일경제,"두산에너빌리티, 국제핵비확산협력센터와 원전 수출 사업지원 협약","\n 원전, SMR 해외시장 진출 관련 정책 개발 환경 조성 사진 확대...",2024-12-27 08:39:43


In [13]:
# 날짜는 파이썬 표준라이브러리 datetime 사용
import datetime
# 오늘 날짜 구하기
today = datetime.datetime.today()
today = today.strftime('%Y-%m-%d')
today

'2024-12-27'

In [14]:
df.to_csv(f'news_{today}.csv',index=False)